In [ ]:
import numpy as np
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt

In [ ]:
# загрузка фрагментов крупномасштабного изображения
datas = np.load('PATH_to_imgs')
masks = np.load('PATH_to_masks')
data = DataLoader(list(zip(datas, masks)),
                     batch_size=8, shuffle=False,
                     pin_memory=True)

In [ ]:
# загрузка модели
# в случае ансамбля потребуется загрузить все модели и расчитать веса (ensembles.ipynb)
model = torch.jit.load('PATH_to_MODEL')
model.eval()

In [ ]:
# получение выходов модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
preds = []
for x, y in data:
  x = x.to(device)
  softmax = nn.Softmax(dim=1)
  predso = torch.argmax(softmax(model(x)),axis=1).to('cpu')
  for i in range(8):
    preds.append(np.array(predso[i,:,:]))

In [ ]:
# функция перевода выходов в 3-канальное изображение
def three(image, class_num, black_color = 0.5):
  pic = np.array(image)
  pic = pic[:, :, np.newaxis]
  img = np.zeros((pic.shape[0], pic.shape[1], class_num))
  np.place(img[ :, :, 0], pic[ :, :, 0] >= black_color, 255)
  np.place(img[ :, :, 1], pic[ :, :, 0] >= black_color, 255)
  np.place(img[ :, :, 2], pic[ :, :, 0] >= black_color, 255)
  img = np.array(img, dtype=np.uint8)
  return img

In [ ]:
# загрузка крупномасштабного изображения и маски
img = cv2.imread('/content/разметка_меньково_2012.jpg')
mask = cv2.imread('/content/разметка_меньково_2012.jpg')
height, width, channels = img.shape

In [ ]:
# объединение preds в выходную крупномасштабную карту
stripes = []
for k in range(height // 256):
  lane = np.zeros((256, 256, 3), dtype=np.uint8)
  for i in range((width // 256)*k, (width // 256)*k+(width // 256)):
    lane = cv2.hconcat([lane, three(preds[i], 3)])
  lane = lane[0:256, 256:(width+256)]
  stripes.append(lane)
combined_image = np.zeros((256, width, 3), dtype=np.uint8)
for j in range(len(stripes)):
  combined_image = cv2.vconcat([combined_image, stripes[j]])
combined_image = combined_image[256:(height+256), 0:width]

In [ ]:
# визуализация крупномасштабного изображения
plt.imshow(img)

In [ ]:
# визуализация крупномасштабной маски
plt.imshow(mask)

In [ ]:
# визуализация выхода модели(ансамбля)
plt.imshow(combined_image)